# TearSheet.ipynb

- doesn't actually generate a tearsheet like a classic print [S&P or Value Line tearsheet](https://financetrain.com/what-is-a-tear-sheet) on a company
- asks Perplexity a few basic questions about the company
- gets info from AlphaVantage
- opens a bunch of browser tabs from major services about the company
- could potentially save those and scrape them to make a tearsheet.


Updated version
- profile - get info from perplexity , other company profiles, openbb, sec filings, wikipedia generate a description of the company, focus on business model history of the company
- news
  - get latest news
  - scrape news using scraper - grab headlines matching pattern
  - filter stuff by how relevant to the company
- chart
- key ratios, top holders
- generate tear sheet using tools
- question answering chatbot using tools


In [2]:
import dotenv
import sys
import os
import re
from datetime import datetime, timedelta
import time
from typing import Dict, Any
from urllib.parse import urljoin, urlparse

import numpy as np
# should require numpy < 2 for pandas_ta or uses more recent ta-lib module
np.NaN = np.nan

import openbb
from openbb import obb
from openbb_core.app.model.obbject import OBBject

import pandas as pd
import pandas_ta as ta

import requests
from bs4 import BeautifulSoup
import json
import aiohttp

import openai
from openai import OpenAI

import IPython
from IPython.display import HTML, Image, Markdown, display

from selenium import webdriver
from selenium.webdriver.common.by import By
# use firefox because it updates less often, can disable updates
# recommend importing profile from Chrome for cookies, passwords
# looks less like a bot with more user cruft in the profile
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service

import wikipedia
import langchain
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

from newsapi import NewsApiClient

from dotenv import load_dotenv

# %pip install mem0ai 
import mem0
from mem0 import MemoryClient

# brew install ta-lib
# https://ta-lib.org/

In [2]:
dotenv.load_dotenv()


True

In [3]:
company = "Tesla"
symbol = "TSLA"


In [4]:
system_prompt = """
You will act as a securities analyst and investment advisor with deep knowledge of financial markets, 
securities analysis, portfolio management. You will maintain a professional yet engaging tone, 
in the style of a senior investment bank research analyst."
"""

user_prompt = f"""You will focus on {company} ({symbol}), and provide a comprehensive analysis covering the following aspects:

Company Profile: An overview of {company}, including its lines of business, history, and recent key developments.

Major News: Significant events related to {company} or its industry impacting its stock.

Financial Performance: Recent earnings reports and stock performance compared to expectations, changes in dividends or stock buybacks.

Analyst Coverage: summarize recent changes to analysts' ratings noting which analyst and firms made upgrades or downgrades; summarize any recent short seller reports noting the firm and analyst.

Product Announcements: Launch of new products, strategic initiatives, or restructurings.

Strategic Moves: Information on deals, partnerships, mergers, acquisitions, divestitures, joint ventures, and major new business and revenue.

Securities Offerings: Announcements related to stock or bond issuances, buybacks, special dividends, or stock splits.

Management Changes: Significant personnel changes within {company}.

Stock Price Movements: Notable stock price changes and their reasons.

Timeline and Future Outlook: A timeline of these events """


In [5]:
def replace_citations(text, citations):
    def replace_match(match):
        citation_num = int(match.group(1))
        if citation_num <= len(citations):
            url = citations[citation_num - 1]  # Citations are 0-indexed, references are 1-indexed
            return f" [{citation_num}]({url})"
        return match.group(0)  # Return original if citation not found
    
    return re.sub(r'\[(\d+)\]', replace_match, text)


In [74]:
perplexity_url = "https://api.perplexity.ai/chat/completions"
perplexity_model = "sonar-pro"

payload = {
    "model": perplexity_model,
    "messages": [
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]
}

perplexity_headers = {
    "Authorization": f"Bearer {os.getenv('PERPLEXITY_API_KEY')}",
    "accept": "application/json",
    "content-type": "application/json"
}

response = requests.post(perplexity_url, json=payload, headers=perplexity_headers)
response_data = response.json()

response_str = response_data['choices'][0]['message']['content']
response_str = response_str.replace("$", "\\\$")
citations = response_data.get('citations', [])
response_str = replace_citations(response_str, citations)

display(Markdown(response_str))


<>:28: DeprecationWarning: invalid escape sequence '\$'
<>:28: DeprecationWarning: invalid escape sequence '\$'
/var/folders/6d/3xz907yn5ylg43s2vlnnzptr0000gn/T/ipykernel_76094/36090712.py:28: DeprecationWarning: invalid escape sequence '\$'
  response_str = response_str.replace("$", "\\\$")


## Notable Tesla (TSLA) News Stories (2024–2025)

### **1. Elon Musk vs. Donald Trump Feud and Impact on Tesla Stock**
- **Date:** June 13, 2025
- **Media:** Investopedia
- **Summary:** A very public feud between Tesla CEO Elon Musk and former President Donald Trump resulted in significant volatility for TSLA shares. The dispute peaked when Musk posted, then deleted, a provocative accusation regarding Trump, which contributed to Tesla losing over \\$150 billion in market capitalization in a single day. Despite the high-profile spat, Tesla shares rebounded the following week as tensions cooled, though the stock remained down 20% for 2025. This episode underscores the sensitivity of Tesla's valuation to Musk's public persona and social media activity [1](https://www.investopedia.com/tesla-stock-overcomes-much-of-the-damage-from-musk-trump-fight-11754295).

### **2. Sales Decline and Financial Performance**
- **Date:** June 2025 (exact date not specified)
- **Media:** CBS News
- **Summary:** Tesla saw a reported 13% drop in sales from the previous year, marking its worst quarter in recent memory. The decline coincided with broader industry challenges and internal controversies, affecting investor sentiment and raising questions about Tesla’s growth trajectory [4](https://www.cbsnews.com/tag/tesla/).

### **3. Focus Shift: Autonomous Vehicles and Optimus Robots**
- **Date:** June 7, 2025
- **Media:** YouTube (analyst coverage)
- **Summary:** Commentary and analysis this year have highlighted Tesla’s heightened emphasis on autonomous vehicle technology and the upcoming Optimus humanoid robots. The strategic pivot is pivotal for the company’s long-term outlook. Tesla investors and market watchers are closely watching whether the company can deliver on these ambitious promises, especially as traditional EV sales growth moderates [3](https://www.youtube.com/watch?v=nZsxOqhy3rc).

### **4. Historical Management Turmoil and Leadership Evolution**
- **Date:** June 2025 (content evergreen, latest review)
- **Media:** Investopedia
- **Summary:** A profile recounts Tesla's early leadership struggles, including Elon Musk’s rise to CEO in 2008 after a series of internal disputes and executive departures. This backstory remains relevant, providing context for Musk’s ongoing dominant influence over the company's strategy and culture, and the historical volatility that continues to shape the company's public image and investor perception [5](https://www.investopedia.com/articles/personal-finance/061915/story-behind-teslas-success.asp).

### **5. Quarterly Financial Results**
- **Date:** Q3 2024 (posted in late 2024)
- **Media:** Tesla Investor Relations
- **Summary:** Tesla released its financial results for the third quarter of 2024. While details are not specified in the search extract, these releases are closely analyzed by the market for insights into operational performance, margin trends, and outlook updates [2](https://ir.tesla.com/press).

---

## **Summary Table: Major Tesla News (2024–2025)**

| Date           | Story / Topic                        | Media           | Key Details                                                                  |
|----------------|-------------------------------------|-----------------|------------------------------------------------------------------------------|
| June 13, 2025  | Musk-Trump Feud Impacts TSLA Stock  | Investopedia    | \\$150B market cap drop; shares later rebounded [1](https://www.investopedia.com/tesla-stock-overcomes-much-of-the-damage-from-musk-trump-fight-11754295).                            |
| June 2025      | Tesla Sales Down 13% YoY            | CBS News        | Worst quarter in recent history for Tesla sales [4](https://www.cbsnews.com/tag/tesla/).                          |
| June 7, 2025   | Focus on Automation/Optimus Robot   | YouTube         | Investor focus shifting to autonomous tech and robotics [3](https://www.youtube.com/watch?v=nZsxOqhy3rc).                  |
| June 2025      | Management History & Profile        | Investopedia    | Review of Musk’s rise and Tesla's early leadership drama [5](https://www.investopedia.com/articles/personal-finance/061915/story-behind-teslas-success.asp).                 |
| Q3 2024        | Earnings Report                     | Tesla IR        | Financial results for Q3 2024 published; market closely studies results [2](https://ir.tesla.com/press).  |

---

## **Conclusion**
Over the past year, Tesla has made headlines due to CEO Elon Musk’s personal controversies, significant volatility in its share price, a notable decline in vehicle sales, and a strategic pivot toward advanced automation and robotics. In addition, analyst coverage and evergreen profiles continue to emphasize the long-running impact of Musk’s dominant role and Tesla’s history of executive upheaval. Each of these stories was reported by respected media outlets or directly via Tesla’s investor communications [1](https://www.investopedia.com/tesla-stock-overcomes-much-of-the-damage-from-musk-trump-fight-11754295) [4](https://www.cbsnews.com/tag/tesla/) [3](https://www.youtube.com/watch?v=nZsxOqhy3rc) [5](https://www.investopedia.com/articles/personal-finance/061915/story-behind-teslas-success.asp) [2](https://ir.tesla.com/press).

In [7]:
perplexity_url = "https://api.perplexity.ai/chat/completions"
perplexity_model = "sonar-pro"  # Changed from sonar-reasoning-pro to sonar-pro
payload = {
    "model": perplexity_model,
    "messages": [
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ],
    "return_citations": True,  # Enable citations for markdown links
    "return_images": True,    # Optional: disable images if not needed
    "return_related_questions": False  # Optional: disable related questions
}
perplexity_headers = {
    "Authorization": f"Bearer {os.getenv('PERPLEXITY_API_KEY')}",
    "accept": "application/json",
    "content-type": "application/json"
}
response = requests.post(perplexity_url, json=payload, headers=perplexity_headers)
response_data = response.json()
response_str = response_data['choices'][0]['message']['content']

# Clean up the response for better markdown rendering
response_str = response_str.replace("$", "\\$")  # Escape dollar signs
citations = response_data.get('citations', [])
response_str = replace_citations(response_str, citations)
images = response_data.get('images', [])
if images:
    image_str = f"![Image 1]({images.pop(0)['image_url']})\n\n" 
    response_str = image_str + response_str
    
display(Markdown(response_str))

![Image 1](https://assets.finbold.com/uploads/2024/10/Heres-Tesla-TSLA-big-secret-for-2025-1024x642.jpg)

## Company Profile

Tesla, Inc. is a leading global manufacturer of electric vehicles (EVs) and energy generation and storage solutions. The company operates in two primary segments:

- Automotive: Tesla designs, develops, manufactures, and sells EVs, offering a range of sedans and SUVs. It also sells automotive regulatory credits, provides after-sales service, insurance, and operates a vast Supercharger network. The automotive business includes direct and used vehicle sales, leasing, purchase financing, and in-app upgrades.
- Energy Generation and Storage: Tesla designs, manufactures, installs, and sells solar energy systems and energy storage products for residential, commercial, and utility customers. The company also offers installation, maintenance, financing, and repair services for these products [2](https://stockanalysis.com/stocks/tsla/company/) [4](https://fortune.com/company/tesla/).

Founded in 2003 and headquartered in Austin, Texas, Tesla has grown into the largest maker of electric cars in the U.S. and, alongside BYD, one of the two largest globally. Elon Musk, who became the largest shareholder in 2004, remains the CEO [2](https://stockanalysis.com/stocks/tsla/company/) [4](https://fortune.com/company/tesla/). As of April 2025, Tesla employs approximately 125,665 people and has over 3.2 billion shares outstanding [3](https://www.sec.gov/Archives/edgar/data/1318605/000162828025018911/tsla-20250331.htm) [4](https://fortune.com/company/tesla/).

## Major News

- Over the past year, Tesla has been impacted by shifting market dynamics in the EV sector, notably increased competition from both traditional automakers and Chinese EV players like BYD [4](https://fortune.com/company/tesla/).
- Challenges in the EV market, including moderating demand growth and price competition, have weighed on Tesla's outlook.
- Regulatory environments in the U.S., China, and Europe remain highly relevant for incentives and expansion.

## Financial Performance

- For the most recent fiscal year, Tesla reported revenues of \$97.7 billion and net profits of \$7.1 billion [4](https://fortune.com/company/tesla/).
- Q1 2025 financial results were recently released, though the specific numbers are not presented in the extract. However, Tesla’s earnings and deliveries have faced downward pressure due to a more competitive market and slower EV adoption in key geographies [1](https://ir.tesla.com) [4](https://fortune.com/company/tesla/).
- There have been no reported changes to Tesla’s dividend policy, which remains at zero, nor have there been any major stock buyback announcements in the latest period [1](https://ir.tesla.com) [3](https://www.sec.gov/Archives/edgar/data/1318605/000162828025018911/tsla-20250331.htm).

## Analyst Coverage

- Tesla remains one of the most actively covered stocks on Wall Street, with analysts from major brokerage firms frequently revising ratings based on delivery numbers, margin trends, and guidance changes. Specific recent upgrades, downgrades or new short seller reports are not listed in the provided results, but Tesla has historically been a target for both bullish and critical analysts.
- The company’s valuation, growth prospects, and competitive risks have been the focus of recent analyst debates, especially given the mixed financial results and market challenges over the last year.

## Product Announcements

- Tesla continues to advance its core vehicle lineup, autonomous driving software, and energy products. The company is known for frequent software updates and incremental design changes across its Model S, 3, X, Y, Cybertruck, and energy products [2](https://stockanalysis.com/stocks/tsla/company/) [5](https://www.morningstar.com/stocks/xnas/tsla/quote).
- No major new mass-market product launches or restructurings are detailed in the recent results, but Tesla’s pipeline includes ongoing work on next-generation vehicle platforms and energy storage solutions.

## Strategic Moves

- While the specific search results do not highlight recent M&A, partnerships, or large-scale joint ventures, Tesla’s strategy continues to emphasize vertical integration, international growth—especially in China and Europe—and technological leadership across EVs, batteries, and solar energy [2](https://stockanalysis.com/stocks/tsla/company/) [4](https://fortune.com/company/tesla/) [5](https://www.morningstar.com/stocks/xnas/tsla/quote).
- Strategic capital allocation has focused on factory buildouts—especially in Austin, Texas, and international locations—alongside autonomous driving development.

## Securities Offerings

- As of mid-April 2025, Tesla’s outstanding share count stood at over 3.2 billion shares [3](https://www.sec.gov/Archives/edgar/data/1318605/000162828025018911/tsla-20250331.htm). There have been no recent public announcements of major secondary offerings, buybacks, special dividends, or stock splits [1](https://ir.tesla.com) [3](https://www.sec.gov/Archives/edgar/data/1318605/000162828025018911/tsla-20250331.htm).
- Tesla’s capital structure remains equity-heavy, with relatively little traditional dividend or buyback activity.

## Management Changes

- Elon Musk continues as CEO. No significant changes to top executive leadership are detailed in these sources for the recent period [2](https://stockanalysis.com/stocks/tsla/company/) [4](https://fortune.com/company/tesla/).

## Stock Price Movements

- Tesla’s market valuation was approximately \$834 billion as of the most recent fiscal year [4](https://fortune.com/company/tesla/).
- The stock price has been volatile, reflecting the broader EV sector’s uncertainties, Tesla’s delivery and margin performance, and shifting expectations for growth and profitability [5](https://www.morningstar.com/stocks/xnas/tsla/quote).
- Downward pressure in 2024 and early 2025 has primarily stemmed from slower-than-expected EV adoption, competition, price cuts, and macroeconomic factors.

## Timeline and Future Outlook

| Date          | Event/Development                                             |
| ------------- | ------------------------------------------------------------ |
| 2024          | Continued to expand vehicle line-up, incremental updates     |
| Q4 2024       | Released annual earnings report (\$97.7B revenue, \$7.1B net)  |
| Q1 2025       | Released latest quarterly financials, highlighted market challenges and slowing growth in key EV segments [1](https://ir.tesla.com) [4](https://fortune.com/company/tesla/) |
| April 2025    | Share count reported at 3.2B [3](https://www.sec.gov/Archives/edgar/data/1318605/000162828025018911/tsla-20250331.htm)                              |
| 2025+         | Focused on next-gen vehicles, global factory expansion, and ongoing development in energy products and autonomous driving |

**Future Outlook:**  
Tesla’s medium-term outlook is shaped by its ability to navigate a slowing EV adoption curve, respond to intensifying competition (especially from Chinese OEMs), and execute on expanding its software and energy business lines. Key investors will watch for updates on new vehicle platforms, energy product scaling, regulatory changes, and operational efficiency. The company’s valuation remains sensitive to execution risks, headline news, and any strategic pivots.

---

**Summary:**  
Tesla is at a critical juncture, balancing its dominant position in the EV and energy storage markets against intensifying competition and changing market growth dynamics. Financial performance has moderated from peak growth rates, analyst sentiment is mixed, and the stock remains volatile. Strategic execution and innovation in software, energy, and manufacturing will be crucial in shaping Tesla’s next phase.

In [8]:
user_prompt = f"""What are the current analyst ratings on {company} ({symbol})? 
Which short sellers issued reports on {company}, if any? Summarize any notable analyst reports."""

payload = {
    "model": perplexity_model,
    "messages": [
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ],
    "return_citations": True,  # Enable citations for markdown links
    "return_images": True,    # Optional: disable images if not needed
    "return_related_questions": False  # Optional: disable related questions
    
}

response = requests.post(perplexity_url, json=payload, headers=perplexity_headers)
response_data = response.json()
response_str = response_data['choices'][0]['message']['content']
response_str = response_str.replace("$", "\\\$")
citations = response_data.get('citations', [])
response_str = replace_citations(response_str, citations)
images = response_data.get('images', [])
if images:
    image_str = f"![Image 1]({images.pop(0)['image_url']})\n\n" 
    response_str = image_str + response_str
    
display(Markdown(response_str))


![Image 1](https://watcher.guru/news/wp-content/uploads/2025/05/opera_rIAzqheKcm-1024x336.png.webp)

## Current Analyst Ratings on Tesla (TSLA)

- The consensus analyst rating for Tesla (TSLA) is **Hold** [2](https://www.marketbeat.com/stocks/NASDAQ/TSLA/forecast/) [5](https://public.com/stocks/tsla/forecast-price-target).
- Out of 64 analysts (as of June 11, 2025) [5](https://public.com/stocks/tsla/forecast-price-target):
  - 23% recommend Strong Buy
  - 27% recommend Buy
  - 28% recommend Hold
  - 8% recommend Sell
  - 14% recommend Strong Sell

- The **consensus price target** is **\\$309.76** based on ratings from 27 analysts [4](https://next.benzinga.com/quote/TSLA/analyst-ratings).
- The **most bullish price target** is **\\$500** (Wedbush, May 23, 2025), while the **most bearish** is **\\$19.05** (GLJ Research, April 23, 2025) [4](https://next.benzinga.com/quote/TSLA/analyst-ratings).
- The three most recent analyst reports (Piper Sandler, Morgan Stanley, Baird; June 9–11, 2025) give an average price target of **\\$376.67**, suggesting a potential 17.88% upside from the current level [4](https://next.benzinga.com/quote/TSLA/analyst-ratings).

## Recent Analyst Upgrades and Downgrades

- In the past 90 days, TSLA has received 3 upgrades and 2 downgrades from major Wall Street analysts [2](https://www.marketbeat.com/stocks/NASDAQ/TSLA/forecast/).
- Key firms covering TSLA include: Argus, Bank of America, Barclays, Benchmark, BNP Paribas, Canaccord Genuity, Cantor Fitzgerald, Cowen, Deutsche Bank, Guggenheim, HSBC, JPMorgan, Morgan Stanley, Piper Sandler, TD Cowen, Goldman Sachs, UBS, Wedbush, Wells Fargo, and more [2](https://www.marketbeat.com/stocks/NASDAQ/TSLA/forecast/).

## Short Sellers and Notable Bearish Reports

- The most noteworthy bearish analyst is **GLJ Research**, which has set a strikingly low price target of **\\$19.05** (April 23, 2025) [4](https://next.benzinga.com/quote/TSLA/analyst-ratings).
- While there is no explicit mention in these results of activist short sellers (such as those publishing major short reports like Hindenburg Research), GLJ Research's target and bearish stance make it a standout among the bears [4](https://next.benzinga.com/quote/TSLA/analyst-ratings).

## Notable Analyst Reports Summary

- **Wedbush** (Bullish): Maintains a \\$500 price target, the highest among major firms, signifying strong conviction in Tesla’s growth prospects and innovation pace [4](https://next.benzinga.com/quote/TSLA/analyst-ratings).
- **GLJ Research** (Bearish): Sets a \\$19.05 target, reflecting deep skepticism on Tesla’s valuation, fundamentals, or business outlook [4](https://next.benzinga.com/quote/TSLA/analyst-ratings).
- **Recent Consensus**: Piper Sandler, Morgan Stanley, and Baird released reports between June 9–11, 2025, with an average target of \\$376.67, implying moderate bullishness and a forecasted upside of nearly 18% [4](https://next.benzinga.com/quote/TSLA/analyst-ratings).

## Summary Table: Analyst Sentiment Breakdown

| Rating           | % of Analysts |
|------------------|--------------|
| Strong Buy       | 23%          |
| Buy              | 27%          |
| Hold             | 28%          |
| Sell             | 8%           |
| Strong Sell      | 14%          |

## Bottom Line

Tesla maintains a polarized analyst following, with a **Hold** consensus reflecting mixed sentiment. Bulls like Wedbush continue to see significant upside, while firms like GLJ Research voice strong concerns. Recent research clusters around a moderate price target, indicating ongoing uncertainty and debate regarding Tesla’s valuation and growth trajectory [2](https://www.marketbeat.com/stocks/NASDAQ/TSLA/forecast/) [4](https://next.benzinga.com/quote/TSLA/analyst-ratings) [5](https://public.com/stocks/tsla/forecast-price-target). No prominent new short seller reports outside traditional bearish analysts are noted in the most recent data.

In [9]:
user_prompt = f"""Have there been any notable news stories about {company} ({symbol}) in the past year, including management profiles and investigative reports? 
For any notable stories, provide the date of publication and the media that reported them."""

payload = {
    "model": perplexity_model,
    "messages": [
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ],
    "return_citations": True,  # Enable citations for markdown links
    "return_images": True,    # Optional: disable images if not needed
    "return_related_questions": False  # Optional: disable related questions
    
}

response = requests.post(perplexity_url, json=payload, headers=perplexity_headers)
response_data = response.json()
response_str = response_data['choices'][0]['message']['content']
response_str = response_str.replace("$", "\\\$")
citations = response_data.get('citations', [])
response_str = replace_citations(response_str, citations)
images = response_data.get('images', [])
if images:
    image_str = f"![Image 1]({images.pop(0)['image_url']})\n\n" 
    response_str = image_str + response_str

display(Markdown(response_str))


![Image 1](https://blog.tipranks.com/wp-content/uploads/2024/12/Image-17-12-24-at-14.49.jpeg?utm_source=nasdaq.com&utm_medium=referral)

## Notable Tesla (TSLA) News Stories, Management Profiles, and Investigative Reports (Past Year)

**Tesla has been a focal point of financial media and broader news coverage throughout the past year, with developments spanning financial performance, executive actions, product launches, and labor relations. Below are the most notable stories with their publication dates and media outlets.**

---

### **Major News & Investigative Reports**

- **Quarterly Earnings Miss and CEO Commitments**
  - *Date:* April 22, 2025
  - *Media:* Business Insider
  - Tesla reported quarterly earnings that missed Wall Street expectations, with revenue of \\$19.34 billion (forecast: \\$21.43 billion) and EPS down 71% year-over-year. Elon Musk addressed ongoing uncertainty about his dual roles and announced a commitment to focus more on Tesla, allocating "far more" of his time to the company following his government advisory work. The earnings call also discussed softening vehicle deliveries, vandalism impacting sales in certain markets, and the company’s retreat from its 2025 "return to growth" forecast. JPMorgan issued a warning about "unprecedented brand damage" in light of these issues [5](https://www.businessinsider.com/tesla-earnings-call-report-tsla-stock-live-updates-2025-4).

- **Sales Decline and Market Performance**
  - *Date:* (Past Year Summary, Highlighted in Recent Stories)
  - *Media:* CBS News
  - Tesla reported a 13% year-over-year drop in sales, marking its worst quarter in recent history. The report situates this decline against broader discussions of labor market softness and macroeconomic headwinds, including political headlines involving Elon Musk [3](https://www.cbsnews.com/tag/tesla/).

- **Product and Technology Shifts**
  - *Date:* October 27, 2024 (and ongoing coverage)
  - *Media:* Tesla Blog, Investor Relations
  - Tesla released its "Master Plan Part 3", outlining a path toward a sustainable energy economy. The company also addressed rumors and reports about employee terminations allegedly linked to union activity, calling such allegations false. This period also included ongoing promotion and updates around vehicle safety milestones and new product launches, such as the Optimus robot and refreshed Model Y [1](https://ir.tesla.com/press) [4](https://www.tesla.com/blog).

---

### **Management Profiles**

- **Elon Musk’s Shifting Focus**
  - *Date:* April 22, 2025
  - *Media:* Business Insider (Earnings Call Coverage)
  - Elon Musk publicly committed to increasing his time at Tesla, citing completion of his main government advisory responsibilities. This move follows months of investor concern over divided attention and its impact on corporate strategy [5](https://www.businessinsider.com/tesla-earnings-call-report-tsla-stock-live-updates-2025-4).

---

### **Labor Relations & Internal Controversy**

- **Alleged Union-Related Terminations**
  - *Date:* October 27, 2024
  - *Media:* Tesla Blog
  - Tesla published a statement refuting investigative reports and social media allegations that claimed employee terminations were retaliation for union drives, labeling such reports as "false allegations." [4](https://www.tesla.com/blog)

---

### **Industry and Vehicle News Highlights**

- **Model Y Achievements**
  - *Date:* June 2025 (Summary from Shareholder and Analyst Discussion)
  - *Media:* YouTube Analyst Review
  - The Model Y was noted as the bestselling vehicle globally for two consecutive years. Attention is now shifting to Tesla's investments in autonomous vehicles and the Optimus humanoid robot, with investor focus on whether Tesla can deliver on these high-profile promises [2](https://www.youtube.com/watch?v=nZsxOqhy3rc).

---

## **Summary Table: Key Tesla News and Reports (June 2024–June 2025)**

| Date          | Outlet             | Headline/Topic                             | Key Points/Implications                                         |
|---------------|---------------------|--------------------------------------------|-----------------------------------------------------------------|
| Apr 22, 2025  | Business Insider    | Earnings Miss, Musk Focus Announcement     | Revenue/EPS miss, Musk commits more time to Tesla, brand risk   |
| 2024–2025     | CBS News            | Sales Drop and Worst Quarter               | 13% sales drop, worst quarter, Musk–Trump/political headlines   |
| Oct 27, 2024  | Tesla Blog          | Master Plan Part 3, Labor Controversy      | Sustainability vision, rebuttal of union-related firing claims  |
| June 2025     | YouTube             | Model Y and Tech Shifts                    | Model Y dominance, shift to autonomy/robotics focus             |

---

### **Conclusion**

In the past year, Tesla attracted significant media attention for:
- Financial and operational challenges, especially missed earnings and sales drops,
- Leadership focus, as CEO Elon Musk announced renewed attention to Tesla after government work,
- Public labor disputes and rebutted claims of union-related firings,
- Ambitious product and technology roadmaps, including autonomous vehicles and robotics.

These stories have been covered by major outlets such as Business Insider, CBS News, and Tesla’s own communications, reflecting high investor, consumer, and regulatory scrutiny on the company [5](https://www.businessinsider.com/tesla-earnings-call-report-tsla-stock-live-updates-2025-4) [3](https://www.cbsnews.com/tag/tesla/) [4](https://www.tesla.com/blog).

In [10]:
# langchain tool

wikipedia_run = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
results = wikipedia_run.run(company)

results


/opt/anaconda3/envs/openbb_agent2/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /opt/anaconda3/envs/openbb_agent2/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


"Page: Nikola Tesla\nSummary: Nikola Tesla (; Serbian Cyrillic: Никола Тесла [nǐkola têsla]; 10 July 1856 – 7 January 1943) was a Serbian-American engineer, futurist, and inventor. He is known for his contributions to the design of the modern alternating current (AC) electricity supply system.\nBorn and raised in the Austrian Empire, Tesla first studied engineering and physics in the 1870s without receiving a degree. He then gained practical experience in the early 1880s working in telephony and at Continental Edison in the new electric power industry. In 1884 he immigrated to the United States, where he became a naturalized citizen. He worked for a short time at the Edison Machine Works in New York City before he struck out on his own. With the help of partners to finance and market his ideas, Tesla set up laboratories and companies in New York to develop a range of electrical and mechanical devices. His AC induction motor and related polyphase AC patents, licensed by Westinghouse Ele

In [11]:
result = wikipedia.search(company)
result


['Nikola Tesla',
 'Tesla, Inc.',
 'Tesla',
 'Tesla Cybertruck',
 'Tesla Model 3',
 'History of Tesla, Inc.',
 'Tesla Model Y',
 'Tesla Roadster (first generation)',
 'Tesla Model S',
 'Tesla coil']

In [12]:
# prompt, I am looking for the wikipedia page of company (ticker). from the list of wikipedia articles, tell me 
# the title of the page that is most likely the best matching wikipedia page for company (ticker). 
# return a page from this list exactly without modification

In [13]:
search_term = 'Tesla, Inc.',
page_object = wikipedia.page(title=search_term, auto_suggest=False)

# printing title
print(page_object.original_title)

# printing links on that page object
#print(page_object.links[0:10])

# printing html of page_object
display(HTML(page_object.html()))


('Tesla, Inc.',)


In [14]:

obb.account.login(email=os.environ['OPENBB_USER'], password=os.environ['OPENBB_PW'], remember_me=True)


In [15]:
obj = obb.equity.fundamental.filings(symbol, form_type='10-K')
r = obj.results[0]
latest_10k_url = r.report_url
latest_10k_url


'https://www.sec.gov/Archives/edgar/data/1318605/000162828025003063/tsla-20241231.htm'

In [16]:
class SECItem1Extractor:
    def __init__(self, file_path_or_url):
        """
        Initialize with either a local file path or URL
        """
        if file_path_or_url.startswith('http'):
            self.content = self._fetch_url(file_path_or_url)
        else:
            with open(file_path_or_url, 'r', encoding='utf-8') as f:
                self.content = f.read()
        
        self.soup = BeautifulSoup(self.content, 'html.parser')
    
    def _fetch_url(self, url):
        """Fetch content from URL with proper headers"""
        headers = {
            'User-Agent': 'streeteye.com druce@streeteye.com',
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.text
    
    def extract_item1(self):
        """
        Extract Item 1 content using multiple strategies
        """
        strategies = [
            self._extract_by_anchor,
            self._extract_by_toc_link,
            self._extract_by_heading_pattern,
            self._extract_by_div_structure
        ]
        
        for strategy in strategies:
            try:
                result = strategy()
                if result and len(result.strip()) > 100:  # Ensure we got substantial content
                    return {
                        'method': strategy.__name__,
                        'content': result,
                        'word_count': len(result.split())
                    }
            except Exception as e:
                print(f"Strategy {strategy.__name__} failed: {e}")
                continue
        
        return None
    
    def _extract_by_anchor(self):
        """Strategy 1: Look for anchor tags that mark Item 1"""
        # Common anchor patterns for Item 1
        anchor_patterns = [
            r'item\s*1(?![0-9a-z])',  # Item 1 not followed by numbers/letters
            r'business',
            r'item1',
            r'i1'
        ]
        
        for pattern in anchor_patterns:
            anchors = self.soup.find_all('a', {'name': re.compile(pattern, re.IGNORECASE)})
            if not anchors:
                anchors = self.soup.find_all('a', {'id': re.compile(pattern, re.IGNORECASE)})
            
            for anchor in anchors:
                content = self._extract_content_after_element(anchor, stop_patterns=[
                    r'item\s*1a', r'item\s*2', r'risk\s*factors'
                ])
                if content:
                    return content
        
        return None
    
    def _extract_by_toc_link(self):
        """Strategy 2: Find Item 1 link in table of contents and follow it"""
        # Look for table of contents links
        toc_links = self.soup.find_all('a', href=True)
        
        item1_pattern = re.compile(r'item\s*1(?![0-9a-z])', re.IGNORECASE)
        
        for link in toc_links:
            if item1_pattern.search(link.get_text()):
                href = link.get('href')
                if href.startswith('#'):
                    # Find the target element
                    target_id = href[1:]
                    target = self.soup.find(id=target_id) or self.soup.find(attrs={'name': target_id})
                    if target:
                        content = self._extract_content_after_element(target, stop_patterns=[
                            r'item\s*1a', r'item\s*1b', r'item\s*2', 
                        ])
                        if content:
                            return content
        
        return None
    
    def _extract_by_heading_pattern(self):
        """Strategy 3: Look for Item 1 heading patterns"""
        # Look for various heading formats
        heading_tags = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'div', 'span']
        item1_pattern = re.compile(r'^[\s]*item\s*1[\s]*[\.:]?[\s]*(?:business|general)?[\s]*$', re.IGNORECASE)
        
        for tag_name in heading_tags:
            elements = self.soup.find_all(tag_name)
            for element in elements:
                text = element.get_text().strip()
                if item1_pattern.match(text):
                    content = self._extract_content_after_element(element, stop_patterns=[
                        r'item\s*1a', r'item\s*2', r'risk\s*factors'
                    ])
                    if content:
                        return content
        
        return None
    
    def _extract_by_div_structure(self):
        """Strategy 4: Look for div structures that might contain Item 1"""
        # Some filings wrap sections in specific div classes or structures
        possible_containers = self.soup.find_all(['div', 'section'], 
                                                class_=re.compile(r'item|business|section', re.IGNORECASE))
        
        item1_pattern = re.compile(r'item\s*1(?![0-9a-z])', re.IGNORECASE)
        
        for container in possible_containers:
            # Check if this container or its immediate children mention Item 1
            container_text = container.get_text()[:200]  # First 200 chars
            if item1_pattern.search(container_text):
                content = self._clean_text(container.get_text())
                if len(content) > 500:  # Ensure substantial content
                    return self._truncate_at_next_item(content)
        
        return None
    
    def _extract_content_after_element(self, start_element, stop_patterns):
        """Extract content from start_element until we hit a stop pattern"""
        content_parts = []
        current = start_element.next_sibling
        
        stop_regex = re.compile('|'.join(stop_patterns), re.IGNORECASE)
        
        while current:
            if hasattr(current, 'get_text'):
                text = current.get_text().strip()
                if text and stop_regex.search(text):
                    break
                content_parts.append(text)
            elif isinstance(current, str):
                text = current.strip()
                if text and stop_regex.search(text):
                    break
                if text:
                    content_parts.append(text)
            
            current = current.next_sibling
            
        content = '\n<~~~>\n'.join(content_parts)
        return self._clean_text(content) if content.strip() else None
    
    def _truncate_at_next_item(self, content):
        """Truncate content at the next item section"""
        stop_patterns = [
            r'item\s*1a',
            r'item\s*2',
            r'risk\s*factors',
            r'unresolved\s*staff\s*comments'
        ]
        
        for pattern in stop_patterns:
            match = re.search(pattern, content, re.IGNORECASE)
            if match:
                return content[:match.start()].strip()
        
        return content
    
    def _clean_text(self, text):
        """Clean up extracted text"""
        # Remove excessive whitespace
        text = re.sub(r'\s+', ' ', text)
        # add back protected newlines
        text = re.sub(r'<~~~>', '\n\n', text)
        # Remove page numbers and other artifacts
        text = re.sub(r'^\s*\d+\s*$', '', text, flags=re.MULTILINE)
        text = re.sub(r'\n\s*\d+\s*\n', '\n', text)
        # Clean up line breaks
        text = re.sub(r'\n\s*\n\s*\n+', '\n\n', text)
        text = re.sub(r'\n{3,}', '\n\n', text)
        
        return text.strip()
    
    def get_document_structure(self):
        """Analyze the document structure to help with debugging"""
        structure = {
            'anchors': [],
            'headings': [],
            'toc_links': []
        }
        
        # Find all anchors
        anchors = self.soup.find_all('a', attrs={'name': True}) + self.soup.find_all('a', attrs={'id': True})
        for anchor in anchors:
            name = anchor.get('name') or anchor.get('id')
            structure['anchors'].append({
                'name': name,
                'text': anchor.get_text()[:50]
            })
        
        # Find potential headings
        for tag in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
            headings = self.soup.find_all(tag)
            for heading in headings:
                text = heading.get_text().strip()
                if 'item' in text.lower():
                    structure['headings'].append({
                        'tag': tag,
                        'text': text
                    })
        
        # Find TOC links
        links = self.soup.find_all('a', href=True)
        for link in links:
            if 'item' in link.get_text().lower() and link.get('href', '').startswith('#'):
                structure['toc_links'].append({
                    'text': link.get_text().strip(),
                    'href': link.get('href')
                })
        
        return structure


In [69]:
client = MemoryClient()

messages = [
    {"role": "user", "content": "What is Item 1 from the latest annual report from Tesla (symbol TSLA)"},
    {"role": "assistant", "content": result['content']},
]
client.add(messages, user_id="tearsheet-TSLA", output_format="v1.1")


KeyboardInterrupt: 

In [68]:
messages = [
    {"role": "user", "content": "Thinking of making a sandwich. What do you recommend?"},
    {"role": "assistant", "content": "How about adding some cheese for extra flavor?"},
    {"role": "user", "content": "Actually, I don't like cheese."},
    {"role": "assistant", "content": "I'll remember that you don't like cheese for future recommendations."}
]
client.add(messages, user_id="alex")


/opt/anaconda3/envs/openbb_agent2/lib/python3.11/site-packages/mem0/client/main.py:34: DeprecationWarning: output_format='v1.0' is deprecated therefore setting it to 'v1.1' by default.Check out the docs for more information: https://docs.mem0.ai/platform/quickstart#4-1-create-memories
  return func(*args, **kwargs)


{'results': [{'id': '90aab049-8528-45af-83f7-501e44930d11',
   'event': 'ADD',
   'memory': 'Does not like cheese'}]}

In [70]:
# Example showing location and preference-aware recommendations
query = "I'm craving some pizza. Any recommendations?"
filters = {
    "AND": [
        {
            "user_id": "alex"
        }
    ]
}
client.search(query, version="v2", filters=filters)

[]

In [5]:
import os
import pandas as pd
import numpy as np
import aiohttp
# import talib
from datetime import datetime, timedelta
from typing import Dict, Any


class MarketData:
    """Handles all market data fetching operations."""

    def __init__(self):
        self.api_key = os.getenv("TIINGO_API_KEY")
        if not self.api_key:
            raise ValueError("TIINGO_API_KEY not found in environment")
            
        self.current_symbol = None
        self.current_df = None

        self.headers = {"Content-Type": "application/json", "Authorization": f"Token {self.api_key}"}

    async def get_historical_data(self, symbol: str, lookback_days: int = 730) -> pd.DataFrame:
        """
        Fetch historical daily data for a given symbol.

        Args:
            symbol (str): The stock symbol to fetch data for.
            lookback_days (int): Number of days to look back from today.

        Returns:
            pd.DataFrame: DataFrame containing historical market data.

        Raises:
            ValueError: If the symbol is invalid or no data is returned.
            Exception: For other unexpected issues during the fetch operation.
        """
        
        # cache df        
        if self.current_symbol == symbol and self.current_df:
            return self.current_df
        
        end_date = datetime.now()
        start_date = end_date - timedelta(days=lookback_days)

        url = (
            f"https://api.tiingo.com/tiingo/daily/{symbol}/prices?"
            f'startDate={start_date.strftime("%Y-%m-%d")}&'
            f'endDate={end_date.strftime("%Y-%m-%d")}'
        )

        try:
            async with aiohttp.ClientSession(timeout=aiohttp.ClientTimeout(total=10)) as session:
                async with session.get(url, headers=self.headers) as response:
                    if response.status == 404:
                        raise ValueError(f"Symbol not found: {symbol}")
                    response.raise_for_status()
                    data = await response.json()

            if not data:
                raise ValueError(f"No data returned for {symbol}")

            df = pd.DataFrame(data)
            df["date"] = pd.to_datetime(df["date"])
            df.set_index("date", inplace=True)

            df[["open", "high", "low", "close"]] = df[["adjOpen", "adjHigh", "adjLow", "adjClose"]].round(2)
            df["volume"] = df["adjVolume"].astype(int)
            df["symbol"] = symbol.upper()
            
            self.current_symbol = symbol
            self.current_df = df            

            return df

        except aiohttp.ClientError as e:
            raise ConnectionError(f"Network error while fetching data for {symbol}: {e}")
        except ValueError as ve:
            raise ve  # Propagate value errors (symbol issues, no data, etc.)
        except Exception as e:
            raise Exception(f"Unexpected error fetching data for {symbol}: {e}")


class TechnicalAnalysis:
    """Technical analysis toolkit using TA-Lib for improved performance."""

    @staticmethod
    def add_core_indicators(df: pd.DataFrame) -> pd.DataFrame:
        """Add a core set of technical indicators using TA-Lib."""
        try:
            # Convert to numpy arrays for TA-Lib (required format)
            high = df["high"].values
            low = df["low"].values
            close = df["close"].values
            volume = df["volume"].values

            # Adding trend indicators (Simple Moving Averages)
            df["sma_20"] = talib.SMA(close, timeperiod=20)
            df["sma_50"] = talib.SMA(close, timeperiod=50)
            df["sma_200"] = talib.SMA(close, timeperiod=200)

            # Adding volatility indicators
            df["atr"] = talib.ATR(high, low, close, timeperiod=14)
            
            # Calculate Average Daily Range Percentage manually
            daily_range = df["high"] - df["low"]
            adr = daily_range.rolling(window=20).mean()
            df["adrp"] = (adr / df["close"]) * 100
            
            # Average volume (20-day)
            df["avg_20d_vol"] = df["volume"].rolling(window=20).mean()

            # Adding momentum indicators
            df["rsi"] = talib.RSI(close, timeperiod=14)
            
            # MACD indicator
            macd, macd_signal, macd_hist = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
            df["macd"] = macd
            df["macd_signal"] = macd_signal
            df["macd_histogram"] = macd_hist

            return df

        except KeyError as e:
            raise KeyError(f"Missing column in input DataFrame: {str(e)}")
        except Exception as e:
            raise Exception(f"Error calculating indicators: {str(e)}")

    @staticmethod
    def check_trend_status(df: pd.DataFrame) -> Dict[str, Any]:
        """Analyze the current trend status."""
        if df.empty:
            raise ValueError("DataFrame is empty. Ensure it contains valid data.")

        latest = df.iloc[-1]
        
        # Handle potential NaN values
        macd_bullish = False
        if not pd.isna(latest["macd"]) and not pd.isna(latest["macd_signal"]):
            macd_bullish = latest["macd"] > latest["macd_signal"]
        
        return {
            "above_20sma": latest["close"] > latest["sma_20"] if not pd.isna(latest["sma_20"]) else False,
            "above_50sma": latest["close"] > latest["sma_50"] if not pd.isna(latest["sma_50"]) else False,
            "above_200sma": latest["close"] > latest["sma_200"] if not pd.isna(latest["sma_200"]) else False,
            "20_50_bullish": latest["sma_20"] > latest["sma_50"] if not pd.isna(latest["sma_20"]) and not pd.isna(latest["sma_50"]) else False,
            "50_200_bullish": latest["sma_50"] > latest["sma_200"] if not pd.isna(latest["sma_50"]) and not pd.isna(latest["sma_200"]) else False,
            "rsi": latest["rsi"] if not pd.isna(latest["rsi"]) else 0,
            "macd_bullish": macd_bullish,
        }


# Usage example
async def analyze_symbol(symbol: str):
    """Complete analysis workflow for a given symbol."""
    market_data = MarketData()
    tech_analysis = TechnicalAnalysis()

    df = await market_data.get_historical_data(symbol)
    df = tech_analysis.add_core_indicators(df)
    trend = tech_analysis.check_trend_status(df)
    
    analysis = f"""
Technical Analysis for {symbol}:

Trend Analysis:
- Above 20 SMA: {'✅' if trend['above_20sma'] else '❌'}
- Above 50 SMA: {'✅' if trend['above_50sma'] else '❌'}
- Above 200 SMA: {'✅' if trend['above_200sma'] else '❌'}
- 20/50 SMA Bullish Cross: {'✅' if trend['20_50_bullish'] else '❌'}
- 50/200 SMA Bullish Cross: {'✅' if trend['50_200_bullish'] else '❌'}

Momentum:
- RSI (14): {trend['rsi']:.2f}
- MACD Bullish: {'✅' if trend['macd_bullish'] else '❌'}

Latest Price: ${df['close'].iloc[-1]:.2f}
Average True Range (14): {df['atr'].iloc[-1]:.2f}
Average Daily Range Percentage: {df['adrp'].iloc[-1]:.2f}%
Average Volume (20D): {df['avg_20d_vol'].iloc[-1]:,.0f}
"""
    print(analysis)
    return df, trend

# Example usage:
df, trend = await analyze_symbol("AAPL")

ValueError: TIINGO_API_KEY not found in environment

In [73]:
from openai import OpenAI
from mem0 import Memory

openai_client = OpenAI()
memory = Memory()

def chat_with_memories(message: str, user_id: str = "default_user") -> str:
    # Retrieve relevant memories
    relevant_memories = memory.search(query=message, user_id=user_id, limit=3)
    memories_str = "\n".join(f"- {entry['memory']}" for entry in relevant_memories["results"])

    # Generate Assistant response
    system_prompt = f"You are a helpful AI. Answer the question based on query and memories.\nUser Memories:\n{memories_str}"
    messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": message}]
    response = openai_client.chat.completions.create(model="gpt-4o-mini", messages=messages)
    assistant_response = response.choices[0].message.content

    # Create new memories from the conversation
    messages.append({"role": "assistant", "content": assistant_response})
    memory.add(messages, user_id=user_id)

    return assistant_response

def main():
    print("Chat with AI (type 'exit' to quit)")
    while True:
        user_input = input("You: ").strip()
        if user_input.lower() == 'exit':
            print("Goodbye!")
            break
        print(f"AI: {chat_with_memories(user_input)}")

main()

Chat with AI (type 'exit' to quit)
You: I don't like cheese. what are some substitutes?
AI: If you don't like cheese, there are several substitutes you can consider, depending on the context in which you would use cheese. Here are some alternatives:

1. **Nutritional Yeast**: This is a popular vegan substitute that has a cheesy flavor and can be sprinkled on popcorn, pasta, or salads.

2. **Tofu**: Silken tofu can be blended to create a creamy texture for sauces or dressings. Firm tofu can also be used in savory dishes for added protein.

3. **Cashew Cream**: Soaked and blended cashews can create a creamy cheese-like texture that works well in spreads and sauces.

4. **Hummus**: This chickpea-based spread can replace cheese in wraps or sandwiches, providing creaminess and flavor.

5. **Vegan Cheese**: There are many commercially available vegan cheese products made from nuts, soy, coconut, or various plant ingredients.

6. **Avocado**: Mashed avocado can serve as a creamy alternative t

In [71]:
filters = {
   "AND": [
      {
         "user_id": "alex"
      }
   ]
}

all_memories = client.get_all(version="v2", filters=filters, page=1, page_size=50)

all_memories


In [75]:

def extract(url):
    extractor = SECItem1Extractor(url)
    
    # Analyze document structure first
    print("Document Structure Analysis:")
    structure = extractor.get_document_structure()
    
    print(f"\nFound {len(structure['anchors'])} anchors:")
    for anchor in structure['anchors'][:10]:  # Show first 10
        print(f"  - {anchor['name']}: {anchor['text']}")
    
    print(f"\nFound {len(structure['headings'])} item-related headings:")
    for heading in structure['headings']:
        print(f"  - {heading['tag']}: {heading['text']}")
    
    print(f"\nFound {len(structure['toc_links'])} TOC links:")
    for link in structure['toc_links']:
        print(f"  - {link['text']} -> {link['href']}")
    
    # Extract Item 1
    print("\n" + "="*50)
    print("EXTRACTING ITEM 1")
    print("="*50)
    
    result = extractor.extract_item1()
    if result:
        print(f"Successfully extracted using method: {result['method']}")
        print(f"Word count: {result['word_count']}")
        print(f"First 500 characters:\n{result['content'][:500]}...")
    else:
        print("Failed to extract Item 1 content")
    return result

        
result = extract(latest_10k_url)
print(result['content'])


Document Structure Analysis:

Found 0 anchors:

Found 0 item-related headings:

Found 23 TOC links:
  - Item 1. -> #ie9fbbc0a99a6483f9fc1594c1ef72807_151
  - Item 1A. -> #ie9fbbc0a99a6483f9fc1594c1ef72807_154
  - Item 1B. -> #ie9fbbc0a99a6483f9fc1594c1ef72807_157
  - Item 1 -> #ie9fbbc0a99a6483f9fc1594c1ef72807_160
  - Item 2. -> #ie9fbbc0a99a6483f9fc1594c1ef72807_163
  - Item 3. -> #ie9fbbc0a99a6483f9fc1594c1ef72807_166
  - Item 4. -> #ie9fbbc0a99a6483f9fc1594c1ef72807_130
  - Item 5. -> #ie9fbbc0a99a6483f9fc1594c1ef72807_271
  - Item 6. -> #ie9fbbc0a99a6483f9fc1594c1ef72807_274
  - Item 7. -> #ie9fbbc0a99a6483f9fc1594c1ef72807_247
  - Item 7A. -> #ie9fbbc0a99a6483f9fc1594c1ef72807_109
  - Item 8. -> #ie9fbbc0a99a6483f9fc1594c1ef72807_1148
  - Item 9. -> #ie9fbbc0a99a6483f9fc1594c1ef72807_277
  - Item 9A. -> #ie9fbbc0a99a6483f9fc1594c1ef72807_280
  - Item 9B. -> #ie9fbbc0a99a6483f9fc1594c1ef72807_133
  - Item 9C. -> #ie9fbbc0a99a6483f9fc1594c1ef72807_283
  - Item 10. -> #ie9fbbc0a99a6

In [78]:
print(result['content'])


ITEM 1. BUSINESS 

 Overview 

 We design, develop, manufacture, sell and lease high-performance fully electric vehicles and energy generation and storage systems, and offer services related to our products. We generally sell our products directly to customers, and continue to grow our customer-facing infrastructure through a global network of vehicle showrooms and service centers, Mobile Service, body shops, Supercharger stations and Destination Chargers to accelerate the widespread adoption of our products. We emphasize performance, attractive styling and the safety of our users and workforce in the design and manufacture of our products and are continuing to develop full self-driving technology for improved safety. We also strive to lower the cost of ownership for our customers through continuous efforts to reduce manufacturing costs and by offering financial and other services tailored to our products. 

 Our mission is to accelerate the world’s transition to sustainable energy. We

In [18]:
# AlphaVantage overview
url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={os.environ['ALPHAVANTAGE_API_KEY']}"
r = requests.get(url)
data = r.json()
pd.DataFrame(data, index=[1]).transpose()


,1
Symbol,TSLA
AssetType,Common Stock
Name,Tesla Inc
Description,"Tesla, Inc. is an American electric vehicle an..."
CIK,1318605
Exchange,NASDAQ
Currency,USD
Country,USA
Sector,MANUFACTURING
Industry,MOTOR VEHICLES & PASSENGER CAR BODIES


In [19]:
exchange = data['Exchange']

morningstarmap = {'NYSE': 'xnys',
                  'NASDAQ': 'xnas'
                 }


In [20]:
now = datetime.now()
end_date = now.strftime('%Y%m%dT%H%M')
sdate = now - timedelta(days=2)
# sdate = datetime(now.year, now.month-1, now.day)
start_date = sdate.strftime('%Y%m%dT%H%M')
start_date, end_date

('20250610T2057', '20250612T2057')

In [21]:
url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={symbol}&apikey={os.environ["ALPHAVANTAGE_API_KEY"]}&time_from={start_date}&time_to={end_date}'

r = requests.get(url)
data = r.json()

for item in data['feed']:
    markdown_str = ""
    date_object = datetime.strptime(item['time_published'], "%Y%m%dT%H%M%S")
    display_title = item['title'].replace("$", "\\\$")  # so Markdown doesn't interpret as latex escape
    description = item['summary'].replace("$", "\\\$")
    markdown_str += f"[{str(date_object)} {display_title}]({item['url']})\n {description}"
    display(Markdown(markdown_str))
    

[2025-06-12 23:26:43 'Attack of the Clones': Coinbase Raises Alarm on Risks With Bitcoin Treasury Model](https://decrypt.co/324934/attack-clones-coinbase-risks-bitcoin-treasury-model)
 A Coinbase analyst warned that the trend of publicly traded companies whipping up Bitcoin treasuries could trigger "systemic risks" for all of crypto.

[2025-06-12 23:14:03 Elon Musk Loses \\$64B In 2025, Still Richer Than Major Automakers Combined - Tesla  ( NASDAQ:TSLA ) ](https://www.benzinga.com/news/movers-shakers/25/06/45917407/elon-musk-net-worth-down-64-billion-in-2025-but-tesla-ceo-still-richer-than-value-of-ford-gen)
 Elon Musk remains the world's richest person, but his wealth is down in 2025. A look at how Musk's wealth stacks up compared to companies. Historic Summer Setup: 3 "Power Patterns" Triggering in the next 75 Days - Get The Details Now The world's richest billionaire has taken a hit to his wealth ...

[2025-06-12 22:16:32 'No Kings' Ad From Walmart Heiress Draws MAGA Ire - Walmart  ( NYSE:WMT ) ](https://www.benzinga.com/news/politics/25/06/45916986/walmart-heiress-takes-on-trump-with-no-kings-ad-white-house-calls-it-burning-some-of-her-inheritanc)
 A Walmart heiress could hurt the retailer with her recent political moves. Several Donald Trump allies are calling for a boycott of Walmart. 9 Out of the Last 10 Summers this "Power Pattern" Delivered Winners - Get The Details Now.

[2025-06-12 21:45:03 Tesla  ( TSLA )  Stock Dips While Market Gains: Key Facts](https://www.zacks.com/stock/news/2496953/tesla-tsla-stock-dips-while-market-gains-key-facts)
 In the latest trading session, Tesla (TSLA) closed at \\$319.11, marking a -2.24% move from the previous day.

[2025-06-12 20:36:57 This AI Powerhouse Stock Is Shockingly Cheap - Alphabet  ( NASDAQ:GOOGL ) ](https://www.benzinga.com/trading-ideas/long-ideas/25/06/45915296/this-ai-powerhouse-stock-is-shockingly-cheap)
 Alphabet trades at a steep valuation discount to magnificent seven peers, despite strong AI, cloud, and ad revenue growth. Google stock offers rare AI upside at a bargain, with robust fundamentals and a 14% analyst upside target.

...

[2025-06-11 07:25:51 Elon Musk says he regrets some social media posts he made about Trump](https://www.cnbc.com/2025/06/11/elon-musk-says-he-regrets-some-social-media-posts-during-trump-showdown.html)
 Elon Musk on Wednesday said he regretted some of the social media posts he made last week during an explosive public dispute with U.S. President Donald Trump.

[2025-06-11 01:05:00 The Best EV Stock to Invest \\$1,000 in Right Now](https://www.fool.com/investing/2025/06/10/the-best-ev-stock-to-invest-1000-in-right-now/)
 If you're looking at the electric vehicle ( EV ) space, you have probably examined Tesla very closely. That makes sense, given that the company basically created the EV market, forcing established automakers to take electric vehicles seriously for the first time.But Tesla is a fairly mature ...

[2025-06-11 00:40:09 Elon Musk says Tesla robotaxi rides in Austin 'tentatively' set to begin June 22](https://www.cnbc.com/2025/06/10/elon-musk-says-tesla-robotaxi-rides-in-austin-will-tentatively-begin-june-22.html)
 Tesla CEO Elon Musk said in a post on X on Tuesday that the first self-driving Tesla will reach a customer's house on June 28.

[2025-06-10 22:02:00 Markets Keep Nudging Higher Ahead of Trade Deals, CPI](https://www.zacks.com/stock/news/2493348/markets-keep-nudging-higher-ahead-of-trade-deals-cpi)
 The S&P 500 is now at its highest level since February 20th.

[2025-06-10 20:59:00 News Moving Wall Street: New Fed Chair? Also Tesla Robotaxi Hits Austin](https://www.zacks.com/commentary/2493189/news-moving-wall-street-new-fed-chair-also-tesla-robotaxi-hits-austin)
 From the potential shifts in Fed leadership to the progress of global trade talks to Tesla's latest moves, investors will be monitoring how these dynamic forces impact the current financial landscape.

In [22]:
NEWSAPI_API_KEY = os.environ['NEWSAPI_API_KEY']

page_size = 100
q = company
date_24h_ago = datetime.now() - timedelta(hours=24)
formatted_date = date_24h_ago.strftime("%Y-%m-%dT%H:%M:%S")
print(f"Fetching top {page_size} stories matching {q} since {formatted_date} from NewsAPI")

api = NewsApiClient(api_key=NEWSAPI_API_KEY)
# sources = api.get_sources()
# pd.DataFrame(sources['sources'])

baseurl = 'https://newsapi.org/v2/everything'

# Define search parameters
params = {
    'q': q,
    'from': formatted_date,
    'language': 'en',
    'sortBy': 'relevancy',
    'apiKey': NEWSAPI_API_KEY,
    'pageSize': 100
}

# Make API call with headers and params
response = requests.get(baseurl, params=params, timeout=60)
if response.status_code != 200:
    print('ERROR: API call failed.')
    print(response.text)
else:
    data = response.json()
    newsapi_df = pd.DataFrame(data['articles'])
    
    # Print the articles
    markdown_str = ""
    for row in newsapi_df.itertuples():
#         date_object = datetime.strptime(row.publishedAt, "%Y%m%dT%H%M%S")
        display_title = row.title.replace("$", "\\\$")  # so Markdown doesn't interpret as latex escape
        markdown_str += f"[{row.publishedAt} {display_title}]({row.url}) \n\n"

    display(Markdown(markdown_str))


Fetching top 100 stories matching Tesla since 2025-06-11T20:57:05 from NewsAPI


[2025-06-11T23:18:11Z Tesla delays robotaxi debut in Austin. Here’s what that means for the stock](https://biztoc.com/x/7d00dd37d519947a) 

[2025-06-12T00:24:39Z How Amazon’s Nuclear Deal Could Solve a Touchy Political Problem](https://biztoc.com/x/86ee8197916f582c) 

[2025-06-12T00:36:18Z Nvidia, Samsung Plan Investments in Robotics Startup Skild AI](https://biztoc.com/x/a76f3172d2ac417a) 

[2025-06-11T23:17:58Z Trump: willing to extend July 8 deadline for trade talks, but not likely necessary](https://biztoc.com/x/b98c82f7898dd486) 

[2025-06-11T23:17:43Z Toyota chairman to face scrutiny over \\$33 billion deal at shareholder meeting](https://biztoc.com/x/ced72a524c72eb68) 

[2025-06-11T23:06:57Z Qube Research hunts for crypto quants to work weekends](https://biztoc.com/x/31ebcd5de2028471) 

[2025-06-12T00:36:03Z Immigration Raids Spread to Agriculture as Meat Plant Targeted](https://biztoc.com/x/b7ddb3a9950b7b56) 

[2025-06-11T23:18:02Z UK surveyors report weakest house price growth in nearly a year, RICS survey shows](https://biztoc.com/x/e0a3ec0d49aff9b1) 

[2025-06-11T23:17:40Z Chime IPO Set to Price Tonight, Stock to Trade Tomorrow](https://biztoc.com/x/0f661aab7ee4d224) 

[2025-06-11T23:17:42Z Trump Says He Will Set Unilateral Tariff Rates Within Two Weeks](https://biztoc.com/x/316824935448d670) 

[2025-06-11T23:17:30Z Taiwan's Laster Tech eyes H2 boost from China's auto stimulus, Tesla Model Y deliveries](https://www.digitimes.com/news/a20250610PD246/laster-lighting-tesla-automotive-revenue.html) 

[2025-06-11T21:38:02Z Musk regrets some of his Trump criticisms; says they 'went too far'](https://japantoday.com/category/world/musk-regrets-some-of-his-trump-criticisms-says-they-%27went-too-far%27) 

[2025-06-12T00:23:36Z Musk’s groveling to Trump begins](https://freethoughtblogs.com/singham/2025/06/11/musks-groveling-to-trump-begins/) 

[2025-06-12T00:40:52Z Big Tech Whistles Along While World Rides Trump-Musk Rollercoaster](https://dailycaller.com/2025/06/11/big-tech-whistles-along-while-world-rides-trump-musk-rollercoaster/) 

[2025-06-11T21:59:59Z Stock Market Today: Indexes Mixed On Trump China News, CPI Data; Tesla Rallies On Robotaxi (Live)](https://biztoc.com/x/f8f9184250c25728) 

[2025-06-11T22:00:21Z Bank of China, JPMorgan Among Banks in Argentina \\$2 Billion Repo](https://biztoc.com/x/7f9a121e52f4bd41) 

[2025-06-11T21:59:57Z Fitch Sees Worsening Outlook for North American Companies](https://biztoc.com/x/c6ba13ac597d0cb6) 

[2025-06-11T21:49:02Z Tesla's robotaxi tests are freaking people out in Texas](https://biztoc.com/x/ff9c4c0c35988d6d) 

[2025-06-11T23:06:44Z Philip Hammond named chair of Ventura Capital advisory board](https://biztoc.com/x/38548a1b2da244ad) 

[2025-06-11T23:06:23Z IMF, Ecuador reach agreement on proposed augmentation of \\$1 billion](https://biztoc.com/x/30631a970a3a65f6) 

[2025-06-11T23:06:37Z Chime prices US IPO at \\$27 per share to raise \\$864 million](https://biztoc.com/x/6af677a8b6974a58) 

[2025-06-11T23:29:06Z Nvidia plans to build its first industrial AI cloud platform in Germany for European manufacturers, combining AI with robotics to assist carmakers like BMW](https://biztoc.com/x/fc9e762f36546a3c) 

[2025-06-12T00:02:39Z Trump says he will set trade tariff rates in the next two weeks](https://biztoc.com/x/5de8994553378657) 

[2025-06-12T00:13:48Z Funds Pile Into Southeast Asian Bonds Despite Record Low Yields](https://biztoc.com/x/755fa37d51e3e52a) 

[2025-06-12T00:24:57Z Japan's business mood worsens as US tariff uncertainty takes toll](https://biztoc.com/x/12337afd0b6ebbe2) 

[2025-06-11T22:44:18Z Vance says Trump doesn't want long-term feud with Musk](https://biztoc.com/x/0f49f956e029bedd) 

[2025-06-11T22:33:09Z Hogs Push Higher on Wednesday, as Cash and Cutout Strength Continues](https://biztoc.com/x/e2a23e9a88edc0d4) 

[2025-06-11T22:22:26Z Ivanhoe Restarts Part of Congo Copper Mine Impacted by Flooding](https://biztoc.com/x/973eaff61631992a) 

[2025-06-11T22:33:33Z Soybeans Pull Back Ahead of USDA Data Day](https://biztoc.com/x/e2ab9bc945b137ee) 

[2025-06-11T22:33:04Z Warner Bros. Says to Settle Buyback, Consent Solicitation Early](https://biztoc.com/x/eb129c9f98f6d48b) 

[2025-06-11T22:55:59Z Australia to work closely with US on review of Biden-era submarine pact](https://biztoc.com/x/62d4724de490c9af) 

[2025-06-11T22:55:45Z US Inflation Beats Estimates for a Fourth Month](https://biztoc.com/x/8bc93f8486223bde) 

[2025-06-12T00:47:10Z India Central Bank Sends Conflicting Signals With Surprise Moves](https://biztoc.com/x/9d244f04cddc5981) 

[2025-06-11T22:55:46Z US Energy Secretary Says Oil Output Drop in 2026 Is ‘Unlikely’](https://biztoc.com/x/c550364c30c185bb) 

[2025-06-11T22:55:27Z Adani Plans Airport Unit IPO by 2027, Hastens \\$100 Billion Capex](https://biztoc.com/x/5dfb1b6f10da474c) 

[2025-06-11T22:44:34Z Gut-wrenching stock declines are more common than you might think](https://biztoc.com/x/088bb82db5c4b25e) 

[2025-06-11T22:44:27Z Wheat Closes with Marginally Mixed Action](https://biztoc.com/x/0820ac57d98dbac7) 

[2025-06-11T22:55:30Z Oil Extends Jump as US Pulls Some Embassy Staff From Middle East](https://biztoc.com/x/5109e5dbd5374be8) 

[2025-06-11T22:55:28Z World Bank Lifts Ban on Nuclear Power, Considers Upstream Gas](https://biztoc.com/x/21ff35677de85a0a) 

[2025-06-11T23:51:21Z Oracle Shares Jump as CEO Targets ‘Dramatically Higher’ Revenue Growth](https://biztoc.com/x/9f25c675ce066d92) 

[2025-06-11T23:40:00Z Chime prices IPO at \\$27 a share, above its expected range](https://biztoc.com/x/eefa8fb2d3ae13df) 

[2025-06-12T00:35:58Z Oaktree’s Marks Says Trump Values Unpredictability, Be Cautious](https://biztoc.com/x/90ac7d7a0fd3e255) 

[2025-06-12T00:35:56Z Gold Advances on Middle East Tensions and Trump’s Tariff Pledge](https://biztoc.com/x/4d51be346da78795) 

[2025-06-11T22:10:48Z Moderna Seeks Outside Investors to Fund Select Vaccine Trials](https://biztoc.com/x/a0e518c31b8ab259) 

[2025-06-11T22:00:14Z GameStop hints at its next bitcoin buy. The stock is tumbling](https://biztoc.com/x/1b79f7e17ced81cf) 

[2025-06-11T22:00:15Z Chanos Hits Back Strategy’s Saylor, Calling Him a ‘Salesman’](https://biztoc.com/x/01a737a43e2c193b) 

[2025-06-11T21:59:53Z Cruz Pitches \\$1.1 Trillion Cut to Fed Bank Payments for Trump Tax Bill](https://biztoc.com/x/d3e95c341d694016) 

[2025-06-11T22:33:00Z A majority of six of the 11 judges on Brazil's Supreme Court have voted to hold social media companies liable for some types of users' content seen as illegal](https://biztoc.com/x/dedf0103c6f11b9a) 

[2025-06-11T22:33:18Z US CDC restores jobs for 450 laid-off employees](https://biztoc.com/x/93dfe61ed7b39033) 

[2025-06-11T22:22:19Z Washington Post announces new editor of opinions desk overhauled by Jeff Bezos](https://biztoc.com/x/f4fc5ab784e9b95a) 

[2025-06-11T22:33:20Z BLET union members vote to ratify 5-year deal with CSX](https://biztoc.com/x/b632169b428bee65) 

[2025-06-11T22:32:55Z What’s up with the wacky CBOT corn spreads? -Braun](https://biztoc.com/x/356b40474e394a45) 

[2025-06-11T22:22:15Z Group of Seven Tries to Avoid Trump Conflict by Scrapping Joint Communique](https://biztoc.com/x/b5bcc713eb39fce2) 

[2025-06-11T22:22:13Z Singapore Order Leads Unlicensed Crypto Exchanges to Weigh Exit](https://biztoc.com/x/1baa21098b11b40d) 

[2025-06-11T22:32:58Z Corn Fails to Move Higher on Record Ethanol Production](https://biztoc.com/x/04f07250e129ada5) 

[2025-06-11T22:33:14Z Cattle Closes Under Pressure on Wednesday](https://biztoc.com/x/d95c4c63e04ced46) 

[2025-06-11T22:33:23Z Pentagon cancels procuring M10 Booker combat vehicles due to 'current world events'](https://biztoc.com/x/9c640bc4df877a7a) 

[2025-06-11T23:06:42Z London IPO advisers eye 2026 rebound as stock market shrinks](https://biztoc.com/x/74bed656dae40e1e) 

[2025-06-11T23:28:51Z Frito-Lay lays off manufacturing staff at Southern California facility after 5 decades](https://biztoc.com/x/7c7bf7de4f2c0d79) 

[2025-06-12T00:02:28Z Michael Knaap resigns as CEO of Australia's Monash IVF](https://biztoc.com/x/b9568ed0cc53acc5) 

[2025-06-12T00:02:48Z US regulators push through last-minute delay to new private fund reporting rules](https://biztoc.com/x/dc79ab58b285e94e) 

[2025-06-12T00:13:43Z OpenAI has discussed raising money from Saudi Arabia, Indian investors, The Information reports](https://biztoc.com/x/04ef6da480c3a5bd) 

[2025-06-12T00:13:51Z Asia’s Easing Prices Pave Path for Rate Cuts and US Divergence](https://biztoc.com/x/c9930ac027fcec0d) 

[2025-06-11T22:22:11Z Top Meat Processor JBS Delays Start of New York Trading](https://biztoc.com/x/a72a02f26f05c79a) 

[2025-06-11T22:22:24Z Trans Mountain Eyes Pipeline Capacity Increase by Early 2027](https://biztoc.com/x/b7fb98fe87fa07e9) 

[2025-06-12T00:35:59Z Monash IVF CEO Steps Down After Latest Embryo Transfer Mix-Up](https://biztoc.com/x/ef49f29f4a8e60ac) 

[2025-06-12T00:36:03Z Immigration Raids Spread to Agriculture as Meat Plant Targeted](https://biztoc.com/x/b7ddb3a9950b7b56) 

[2025-06-12T00:13:43Z Warner Bros. Discovery bonds see big selloff as ratings are cut to junk. Should stock investors be worried?](https://biztoc.com/x/78824eace15a28d0) 

[2025-06-12T00:13:24Z Ex-CIA analyst sentenced to over 3 years for leak of classified information](https://biztoc.com/x/0f3c2ab69c34cfbe) 

[2025-06-12T00:13:56Z Japan Credit Sales See Busiest June in Years as Buyers Seek Risk](https://biztoc.com/x/b46d78ae183113c6) 

[2025-06-11T21:59:49Z Social Media Firms Accountable for Posts, Brazil Judges Conclude](https://biztoc.com/x/3620d232566089b6) 

[2025-06-11T22:55:44Z US Senate rejects bids to block arms sales to Qatar, UAE](https://biztoc.com/x/9a2b05081ed652bc) 

[2025-06-11T22:55:59Z Oil Jumps on Mideast Tensions, Stocks to Open Flat: Markets Wrap](https://biztoc.com/x/a6bf62b0205ff0b2) 

[2025-06-11T22:55:37Z Chime, Backers Raise \\$864 Million in Above-Range IPO](https://biztoc.com/x/e0c17843850bec5b) 

[2025-06-11T22:00:18Z Colombia’s Petro Moves Ahead With Labor Referendum Despite Rising Violence](https://biztoc.com/x/8dff84f25e35967f) 

[2025-06-11T22:00:19Z Ex-Columbia Student Khalil Ordered Released From ICE Custody](https://biztoc.com/x/86da27fa06646ece) 

[2025-06-11T22:33:15Z Cotton Closes Mixed on Wednesday](https://biztoc.com/x/6b2aa9d1d3285781) 

[2025-06-11T23:29:13Z CNN Prepares for the Afterlife](https://biztoc.com/x/35c75922a66c5b06) 

[2025-06-11T23:40:05Z UnitedHealth Stock Just Raised Its Dividend. Should You Buy UNH Now?](https://biztoc.com/x/5282b95c419c516f) 

[2025-06-11T23:51:23Z Chicago Bears Weigh Sale of McKenna’s Minority Stake](https://biztoc.com/x/de172ce67cdcde7e) 

[2025-06-11T23:51:12Z UK House Price Indicator Slides to 10-Month Low as Market Cools](https://biztoc.com/x/54f5c16778a339c8) 

[2025-06-12T00:02:27Z Thune and Crapo to huddle with Trump](https://biztoc.com/x/0b0bd6d9525dd566) 

[2025-06-12T00:53:09Z Waymo and Tesla are locked in a life-and-death struggle over who can bring to fruition the worst possible future for self-driving cars in society [Fail]](https://www.fark.com/comments/13702826/Waymo-Tesla-are-locked-in-a-life-death-struggle-over-who-can-bring-to-fruition-worst-possible-future-for-self-driving-cars-in-society) 

[2025-06-11T21:22:03Z US stock market: S&P 500, Dow Jones, Nasdaq, Russell 2000 fall. Check Apple, Tesla, Amazon, Nvidia stocks](https://economictimes.indiatimes.com/news/international/us/us-stock-market-sp-500-dow-jones-nasdaq-russell-2000-fall-check-apple-tesla-amazon-nvidia-stocks/articleshow/121787622.cms) 

[2025-06-12T00:53:01Z US President Donald Trump "acknowledged" Musk's apology, says White House Press Secy](https://economictimes.indiatimes.com/news/international/global-trends/us-president-donald-trump-acknowledged-musks-apology-says-white-house-press-secy/articleshow/121790228.cms) 

[2025-06-11T21:32:49Z Operation Secure: INTERPOL dismantles 20,000+ malicious IPs in major cybercrime crackdown](https://securityaffairs.com/178898/cyber-crime/operation-secure-interpol-dismantles-20000-malicious-ips-in-major-cybercrime-crackdown.html) 

[2025-06-11T21:38:33Z Musk-Trump Truce Sends Dogecoin Skyward, With Ethereum Along for the Ride: Analysis](https://decrypt.co/324715/musk-trump-truce-dogecoin-ethereum-price-analysis) 

[2025-06-11T21:04:25Z Walmart heiress Christy Walton’s anti-Trump ad fuels MAGA-led call for boycott of retailer](https://nypost.com/2025/06/11/business/walmart-heiress-christy-waltons-anti-trump-ad-fuels-maga-led-call-for-boycott-of-retailer/) 

[2025-06-11T23:32:16Z Musk began Trump apology effort after call with Vance, Wiles - Axios](https://slashdot.org/firehose.pl?op=view&amp;id=178014151) 

[2025-06-11T21:52:56Z Elon Musk says some of his social media posts about Trump 'went too far' - NPR](https://slashdot.org/firehose.pl?op=view&amp;id=178013299) 

[2025-06-11T21:39:00Z U.S. stocks dip as investors weigh inflation, China-U.S. trade talks](https://www.thestar.com.my/news/world/2025/06/12/us-stocks-dip-as-investors-weigh-inflation-china-us-trade-talks) 



In [23]:
now = datetime.now()
end_date = now.strftime('%Y-%m-%d')
# start_date = datetime.now() - timedelta(days=30)
sdate = datetime(now.year, now.month-1, now.day)
start_date = sdate.strftime('%Y-%m-%d')
start_date, end_date

('2025-05-12', '2025-06-12')

In [24]:
API_KEY = os.environ['NEWSFILTER_API_KEY']
API_ENDPOINT = "https://api.newsfilter.io/search?token={}".format(API_KEY)

# Define the news search parameters
queryString = f"symbols:{symbol} AND publishedAt:[{start_date} TO {end_date}]"

payload = {
    "queryString": queryString,
    "from": 0,
    "size": 10
}

# Send the search query to the Search API
response = requests.post(API_ENDPOINT, json=payload)

# Read the response
articles = response.json()


# Read the response
articles = response.json()

for item in articles['articles']:
    markdown_str = ""
    date_array = item['publishedAt'].split("T")
    datestr = date_array[0] + "T" + date_array[1][:8]
#     print(item['publishedAt'])
#     print(date_array)
#     print(datestr)
    date_object = datetime.strptime(datestr, "%Y-%m-%dT%H:%M:%S")
    display_title = item['title'].replace("$", "\\\$")  # so Markdown doesn't interpret as latex escape
    description = item['description'].replace("$", "\\\$")

    markdown_str += f"[{str(date_object)} {display_title}]({item['url']})\n {description}"
    display(Markdown(markdown_str))

[2025-06-12 23:05:55 Global EV sales rise in May as China hits 2025 peak -Rho Motion](https://newsfilter.io/articles/global-ev-sales-rise-in-may-as-china-hits-2025-peak--rho-motion-c975e037573f9412f1feeed77f412343)
 Global sales of electric and plug-in hybrid vehicles rose 24% in May compared with the same period a year ago, as strength in China offset slower growth in North America, according to market research firm Rho Motion.

[2025-06-12 17:03:05 144 Form - Report of proposed sale of securities - Tesla, Inc. (0001318605) (Subject)](https://newsfilter.io/articles/144-form---report-of-proposed-sale-of-securities---tesla-inc-0001318605-subject-e2b63d60164b3e41f800322b2c334454)
 Report of proposed sale of securities

[2025-06-12 20:21:17 Tesla to take Japan virtual power plant business nationwide, Nikkei reports](https://newsfilter.io/articles/tesla-to-take-japan-virtual-power-plant-business-nationwide-nikkei-reports-912703a56a409ac85f27dce80abb29cc)
 Tesla will expand its virtual power plant business in Japan by providing free storage batteries to companies and remotely managing them to help balance energy supply and demand, Nikkei reported on Friday.

[2025-06-12 19:07:38 AMD reveals next-generation AI chips with OpenAI CEO Sam Altman](https://newsfilter.io/articles/amd-reveals-next-generation-ai-chips-with-openai-ceo-sam-altman-c3c8d6df8b6fd2d02b54de3e55a2d3ef)
 The MI400 chips will be able to be assembled into a full server rack called Helios.

[2025-06-12 10:19:19 Wall Street futures slip as Middle East tensions rise; Boeing falls](https://newsfilter.io/articles/wall-street-futures-slip-as-middle-east-tensions-rise-boeing-falls-376f4e6d3d3d1693cd9349230cf1ceaf)
 U.S. stock index futures slipped on Thursday as signs of rising tensions in the Middle East weighed on risk sentiment and investors sought more clarity on Washington's recent trade deals with China.

[2025-06-12 01:27:24 Elon Musk's favorability among Republicans dropped 16 points since March, Quinnipiac says](https://newsfilter.io/articles/elon-musks-favorability-among-republicans-dropped-16-points-since-march-quinnipiac-says-97b0090ee80ff9714ea5bd7d40fc98ce)
 The Tesla CEO's favorability scores have declined among GOP voters, according to public opinion survey results from Quinnipiac University.

[2025-06-11 17:34:29 Elon Musk's robotaxi launch in Texas tests his vision of Tesla's future](https://newsfilter.io/articles/elon-musks-robotaxi-launch-in-texas-tests-his-vision-of-teslas-future-4dff31e71e0f7512dc6e4f85fbcaf88b)
 Tesla faces an existential test this month as it aims to launch self-driving “robotaxis” in its home base of Austin, Texas, where public-safety officials are increasingly concerned about the state’s anti-regulation stance toward autonomous vehicles.

[2025-06-11 14:32:43 Tesla gains for a fourth day as Musk sets robotaxi date, Trump tensions cool](https://newsfilter.io/articles/tesla-gains-for-a-fourth-day-as-musk-sets-robotaxi-date-trump-tensions-cool-a3a155c71b01c26972d4a6232993b745)
 Tesla shares rallied for a fourth straight session as the spat between CEO Elon Musk and President Donald Trump showed signs of a cooldown. 

[2025-06-11 11:39:55 Some French Tesla drivers file lawsuit over harm allegedly caused by Musk's behaviour](https://newsfilter.io/articles/some-french-tesla-drivers-file-lawsuit-over-harm-allegedly-caused-by-musks-behaviour-030c4caa6badfddd0231e81177940e5c)
 A small group of Tesla owners in France is suing the carmaker run by Elon Musk, alleging its vehicles have become "far-right totems" that are harming their reputation, the law firm representing the group said on Wednesday.

[2025-06-11 10:33:31 US stock futures dip on lack of details from China trade talk, CPI in focus](https://newsfilter.io/articles/us-stock-futures-dip-on-lack-of-details-from-china-trade-talk-cpi-in-focus-99b2580c565d7d253fed1f8aab0a4bbd)
 U.S. stock index futures edged lower on Wednesday after U.S.-China trade talks offered little sign of a durable resolution to their longstanding trade tensions, while investors turned their focus to a key inflation reading.

In [25]:
# https://sethhobson.com/2025/01/building-a-stock-analysis-server-with-mcp-part-1/ 
    
class MarketData:
    """Handles all market data fetching operations."""

    def __init__(self):
        self.api_key = os.getenv("TIINGO_API_KEY")
        if not self.api_key:
            raise ValueError("TIINGO_API_KEY not found in environment")

        self.headers = {"Content-Type": "application/json", "Authorization": f"Token {self.api_key}"}

    async def get_historical_data(self, symbol: str, lookback_days: int = 365) -> pd.DataFrame:
        """
        Fetch historical daily data for a given symbol.

        Args:
            symbol (str): The stock symbol to fetch data for.
            lookback_days (int): Number of days to look back from today.

        Returns:
            pd.DataFrame: DataFrame containing historical market data.

        Raises:
            ValueError: If the symbol is invalid or no data is returned.
            Exception: For other unexpected issues during the fetch operation.
        """
        end_date = datetime.now()
        start_date = end_date - timedelta(days=lookback_days)

        url = (
            f"https://api.tiingo.com/tiingo/daily/{symbol}/prices?"
            f'startDate={start_date.strftime("%Y-%m-%d")}&'
            f'endDate={end_date.strftime("%Y-%m-%d")}'
        )

        try:
            async with aiohttp.ClientSession(timeout=aiohttp.ClientTimeout(total=10)) as session:
                async with session.get(url, headers=self.headers) as response:
                    if response.status == 404:
                        raise ValueError(f"Symbol not found: {symbol}")
                    response.raise_for_status()
                    data = await response.json()

            if not data:
                raise ValueError(f"No data returned for {symbol}")

            df = pd.DataFrame(data)
            df["date"] = pd.to_datetime(df["date"])
            df.set_index("date", inplace=True)

            df[["open", "high", "low", "close"]] = df[["adjOpen", "adjHigh", "adjLow", "adjClose"]].round(2)
            df["volume"] = df["adjVolume"].astype(int)
            df["symbol"] = symbol.upper()

            return df

        except aiohttp.ClientError as e:
            raise ConnectionError(f"Network error while fetching data for {symbol}: {e}")
        except ValueError as ve:
            raise ve  # Propagate value errors (symbol issues, no data, etc.)
        except Exception as e:
            raise Exception(f"Unexpected error fetching data for {symbol}: {e}")
            

In [26]:
class TechnicalAnalysis:
    """Technical analysis toolkit with improved performance and readability."""

    @staticmethod
    def add_core_indicators(df: pd.DataFrame) -> pd.DataFrame:
        """Add a core set of technical indicators."""
        try:
            # Adding trend indicators
            df["sma_20"] = ta.sma(df["close"], length=20)
            df["sma_50"] = ta.sma(df["close"], length=50)
            df["sma_200"] = ta.sma(df["close"], length=200)

            # Adding volatility indicators and volume
            daily_range = df["high"].sub(df["low"])
            adr = daily_range.rolling(window=20).mean()
            df["adrp"] = adr.div(df["close"]).mul(100)
            df["avg_20d_vol"] = df["volume"].rolling(window=20).mean()

            # Adding momentum indicators
            df["atr"] = ta.atr(df["high"], df["low"], df["close"], length=14)
            df["rsi"] = ta.rsi(df["close"], length=14)
            macd = ta.macd(df["close"], fast=12, slow=26, signal=9)
            if macd is not None:
                df = pd.concat([df, macd], axis=1)

            return df

        except KeyError as e:
            raise KeyError(f"Missing column in input DataFrame: {str(e)}")
        except Exception as e:
            raise Exception(f"Error calculating indicators: {str(e)}")

    @staticmethod
    def check_trend_status(df: pd.DataFrame) -> Dict[str, Any]:
        """Analyze the current trend status."""
        if df.empty:
            raise ValueError("DataFrame is empty. Ensure it contains valid data.")

        latest = df.iloc[-1]
        return {
            "above_20sma": latest["close"] > latest["sma_20"],
            "above_50sma": latest["close"] > latest["sma_50"],
            "above_200sma": latest["close"] > latest["sma_200"],
            "20_50_bullish": latest["sma_20"] > latest["sma_50"],
            "50_200_bullish": latest["sma_50"] > latest["sma_200"],
            "rsi": latest["rsi"],
            "macd_bullish": latest.get("MACD_12_26_9", 0) > latest.get("MACDs_12_26_9", 0),
        }

In [27]:
market_data = MarketData()
tech_analysis = TechnicalAnalysis()


In [28]:
df = await market_data.get_historical_data(symbol)
df

,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor,symbol
date,,,,,,,,,,,,,
2024-06-12 00:00:00+00:00,177.29,180.55,169.80,171.12,90389446,177.29,180.55,169.800,171.120,90389446,0.0,1.0,TSLA
2024-06-13 00:00:00+00:00,182.47,191.08,181.23,188.39,118984122,182.47,191.08,181.230,188.390,118984122,0.0,1.0,TSLA
2024-06-14 00:00:00+00:00,178.01,186.00,176.92,185.80,82038194,178.01,186.00,176.920,185.800,82038194,0.0,1.0,TSLA
2024-06-17 00:00:00+00:00,187.44,188.81,177.00,177.92,109786083,187.44,188.81,177.000,177.920,109786083,0.0,1.0,TSLA
2024-06-18 00:00:00+00:00,184.86,187.20,182.37,186.56,68982265,184.86,187.20,182.370,186.560,68982265,0.0,1.0,TSLA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-06 00:00:00+00:00,295.14,305.50,291.14,298.83,164747685,295.14,305.50,291.140,298.830,164747685,0.0,1.0,TSLA
2025-06-09 00:00:00+00:00,308.58,309.83,281.85,285.96,140908876,308.58,309.83,281.850,285.955,140908876,0.0,1.0,TSLA
2025-06-10 00:00:00+00:00,326.09,327.83,310.67,314.94,151256520,326.09,327.83,310.667,314.940,151256520,0.0,1.0,TSLA


In [29]:
df = tech_analysis.add_core_indicators(df)
df


,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,...,sma_20,sma_50,sma_200,adrp,avg_20d_vol,atr,rsi,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
date,,,,,,,,,,,,,,,,,,,,,
2024-06-12 00:00:00+00:00,177.29,180.55,169.80,171.12,90389446,177.29,180.55,169.800,171.120,90389446,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-13 00:00:00+00:00,182.47,191.08,181.23,188.39,118984122,182.47,191.08,181.230,188.390,118984122,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-14 00:00:00+00:00,178.01,186.00,176.92,185.80,82038194,178.01,186.00,176.920,185.800,82038194,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-17 00:00:00+00:00,187.44,188.81,177.00,177.92,109786083,187.44,188.81,177.000,177.920,109786083,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-18 00:00:00+00:00,184.86,187.20,182.37,186.56,68982265,184.86,187.20,182.370,186.560,68982265,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-06 00:00:00+00:00,295.14,305.50,291.14,298.83,164747685,295.14,305.50,291.140,298.830,164747685,...,335.7815,291.9192,305.07280,5.301213,1.189470e+08,19.109930,41.724119,6.462075,-8.404693,14.866769
2025-06-09 00:00:00+00:00,308.58,309.83,281.85,285.96,140908876,308.58,309.83,281.850,285.955,140908876,...,336.2975,292.6282,305.51020,5.247586,1.193730e+08,19.743506,46.661534,4.567715,-8.239243,12.806958
2025-06-10 00:00:00+00:00,326.09,327.83,310.67,314.94,151256520,326.09,327.83,310.667,314.940,151256520,...,336.6830,293.8790,306.02430,5.064706,1.212945e+08,19.708256,52.328390,4.428285,-6.702938,11.131223


In [30]:
trend = tech_analysis.check_trend_status(df)
analysis = f"""
Technical Analysis for {symbol}:

Trend Analysis:
- Above 20 SMA: {'✅' if trend['above_20sma'] else '❌'}
- Above 50 SMA: {'✅' if trend['above_50sma'] else '❌'}
- Above 200 SMA: {'✅' if trend['above_200sma'] else '❌'}
- 20/50 SMA Bullish Cross: {'✅' if trend['20_50_bullish'] else '❌'}
- 50/200 SMA Bullish Cross: {'✅' if trend['50_200_bullish'] else '❌'}

Momentum:
- RSI (14): {trend['rsi']:.2f}
- MACD Bullish: {'✅' if trend['macd_bullish'] else '❌'}

Latest Price: ${df['close'].iloc[-1]:.2f}
Average True Range (14): {df['atr'].iloc[-1]:.2f}
Average Daily Range Percentage: {df['adrp'].iloc[-1]:.2f}%
Average Volume (20D): {df['avg_20d_vol'].iloc[-1]}
"""
print(analysis)



Technical Analysis for TSLA:

Trend Analysis:
- Above 20 SMA: ❌
- Above 50 SMA: ✅
- Above 200 SMA: ✅
- 20/50 SMA Bullish Cross: ✅
- 50/200 SMA Bullish Cross: ❌

Momentum:
- RSI (14): 49.87
- MACD Bullish: ❌

Latest Price: $319.11
Average True Range (14): 18.98
Average Daily Range Percentage: 5.10%
Average Volume (20D): 118915354.2



In [31]:
# open pages via selenium and firefox
outputdir = "htmldata"

# Print the formatted time
print(datetime.now().strftime('%H:%M:%S'), "Starting", flush=True)

firefox_app_path = '/Applications/Firefox.app'
# Path to your geckodriver
geckodriver_path = '/Users/drucev/webdrivers/geckodriver'
# Set up Firefox options to use your existing profile
# important for some sites that need a login, also a generic profile fingerprint that looks like a bot might get blocked
firefox_profile_path = '/Users/drucev/Library/Application Support/Firefox/Profiles/x6dmpnoo.default-release-1'
options = Options()
options.profile = firefox_profile_path

print(datetime.now().strftime('%H:%M:%S'), "Initialized profile", flush=True)

# Create a Service object with the path
service = Service(geckodriver_path)

print(datetime.now().strftime('%H:%M:%S'), "Initialized service", flush=True)
# Set up the Firefox driver
driver = webdriver.Firefox(service=service, options=options)

print(datetime.now().strftime('%H:%M:%S'), "Initialized webdriver", flush=True)
sleeptime = 10


20:57:05 Starting
20:57:06 Initialized profile
20:57:06 Initialized service
20:57:10 Initialized webdriver


In [32]:
# Open a new tab
def open_tab(driver, url):
    driver.execute_script(f"window.open('{url}');")
    # Switch to last one opened - sometimes hangs?
    # driver.switch_to.window(driver.window_handles[-1])
    # print(url)
    # time.sleep(sleeptime)  


In [33]:
url = f"https://stockcharts.com/h-sc/ui?s={symbol}&id=p33407302522&def=Y&listNum=1#"
source = "Stockcharts"
# Open the page
open_tab(driver, url)

# Wait for the page to load
# time.sleep(sleeptime)  
display(Markdown(f"[{source}]({url})"))

[Stockcharts](https://stockcharts.com/h-sc/ui?s=TSLA&id=p33407302522&def=Y&listNum=1#)

In [34]:
url = f'https://www.bloomberg.com/search?query={company}'
source = "Bloomberg"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[Bloomberg](https://www.bloomberg.com/search?query=Tesla)

In [35]:
url = f'https://www.reuters.com/site-search/?query={company}&offset=0'
source = "Reuters"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[Reuters](https://www.reuters.com/site-search/?query=Tesla&offset=0)

In [36]:
url = f'https://finance.yahoo.com/quote/{symbol}/news'
source = "Yahoo quote"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[Yahoo quote](https://finance.yahoo.com/quote/TSLA/news)

In [37]:
url = f'https://finance.yahoo.com/quote/{symbol}/key-statistics?ltr=1'
source = "Yahoo stats"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[Yahoo stats](https://finance.yahoo.com/quote/TSLA/key-statistics?ltr=1)

In [38]:
url = f'https://www.ft.com/search?q={company}'
source = "FT"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[FT](https://www.ft.com/search?q=Tesla)

In [39]:
url = f'https://www.marketwatch.com/investing/stock/{symbol}?mod=search_symbol'
source = "Marketwatch"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[Marketwatch](https://www.marketwatch.com/investing/stock/TSLA?mod=search_symbol)

In [40]:
url = f'https://www.barrons.com/market-data/stocks/{symbol}?mod=searchresults_companyquotes&mod=searchbar&search_keywords={company}&search_statement_type=suggested'    
source = "Barrons"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[Barrons](https://www.barrons.com/market-data/stocks/TSLA?mod=searchresults_companyquotes&mod=searchbar&search_keywords=Tesla&search_statement_type=suggested)

In [50]:
url = f'https://www.businessinsider.com/answers#{company}'

source = "Insider"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[Insider](https://www.businessinsider.com/answers#Tesla)

In [42]:
url = f'https://www.google.com/finance/quote/{symbol}:{exchange}'
source = "Google Finance"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[Google Finance](https://www.google.com/finance/quote/TSLA:NASDAQ)

In [43]:
url = f'https://finviz.com/quote.ashx?t={symbol}&p=d'
source = "FinViz"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[FinViz](https://finviz.com/quote.ashx?t=TSLA&p=d)

In [44]:
url = f'https://www.reddit.com/search?q={company}&include_over_18=on&sort=relevance&t=all'
source = "Reddit"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[Reddit](https://www.reddit.com/search?q=Tesla&include_over_18=on&sort=relevance&t=all)

In [45]:
morn_exch = morningstarmap[exchange]
url = f'https://www.morningstar.com/stocks/{morn_exch}/{symbol}/quote'
source = "Morningstar"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))
    

[Morningstar](https://www.morningstar.com/stocks/xnas/TSLA/quote)

In [46]:
url = f'https://whalewisdom.com/stock/{symbol}'
source = "WhaleWisdom"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))


[WhaleWisdom](https://whalewisdom.com/stock/TSLA)

In [47]:
url = f'https://www.gurufocus.com/stock/{symbol}/guru-trades'
source = "GuruFocus"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))


[GuruFocus](https://www.gurufocus.com/stock/TSLA/guru-trades)

In [48]:
# https://www.barchart.com/stocks/highs-lows/lows?orderBy=highPercent1y&orderDir=asc
#     https://stockcharts.com/h-sc/ui?s=%24DJUSEN&id=p33407302522&def=Y&listNum=1
#         https://stockcharts.com/h-sc/ui?s=%24DJUSDN&id=p33407302522&def=Y&listNum=1

In [49]:
# driver.close()
